In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from typing import Optional, List, Dict, Any
import pandas as pd
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(".env")

def load_env(env_file: str = "../../.env") -> None:
    """
    Load environment variables from a .env file.
    Works in both script and notebook environments.
    """

    env_path = Path(env_file)

    if not env_path.is_absolute():
        # Script
        if "__file__" in globals():
            project_root = Path(__file__).resolve().parents[1]
        # Notebook
        else:
            project_root = Path.cwd()

        env_path = project_root / env_path

    if not env_path.exists():
        raise FileNotFoundError(f"❌ .env file not found: {env_path}")
    
    # load_dotenv(env_path)

def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Goal
    ----
    Load configuration parameters from a YAML file.

    This function works both in:
    - Python scripts
    - Jupyter notebooks (where __file__ is not defined)

    Parameters
    ----------
    config_filename : str
        Relative or absolute path to the YAML configuration file.

    Returns
    -------
    dict
        Dictionary containing configuration parameters.
    """

    config_path = Path(config_filename)

    # Cas 1 : chemin absolu → on l'utilise directement
    if config_path.is_absolute():
        final_path = config_path

    else:
        # Cas 2 : Script Python (__file__ existe)
        if "__file__" in globals():
            project_root = Path(__file__).resolve().parents[1]

        # Cas 3 : Notebook Jupyter (__file__ n'existe pas)
        else:
            project_root = Path(os.getcwd())

        final_path = project_root / config_path

    if not final_path.exists():
        raise FileNotFoundError(f"❌ Config file not found: {final_path}")

    with open(final_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config


def create_postgres_engine(
    host: str,
    database: str,
    user: str,
    password: str,
    port: int = 5432
) -> Optional[Engine]:
    """
    Crée un moteur SQLAlchemy pour PostgreSQL.

    Paramètres
    ----------
    host : str
        Adresse du serveur PostgreSQL
    database : str
        Nom de la base de données
    user : str
        Utilisateur PostgreSQL
    password : str
        Mot de passe
    port : int, optionnel
        Port PostgreSQL (par défaut 5432)

    Retour
    ------
    sqlalchemy.engine.Engine | None
        Moteur SQLAlchemy ou None en cas d'erreur
    """
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}",
            pool_pre_ping=True
        )
        return engine
    except Exception as e:
        print(f"❌ Erreur de création du moteur SQLAlchemy : {e}")
        return None


def execute_select(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> List[Dict[str, Any]]:
    """
    Exécute une requête SELECT et retourne les résultats sous forme de dictionnaires.
    """
    with engine.connect() as conn:
        result = conn.execute(text(query), params or {})
        return [dict(row._mapping) for row in result]
    
def select_to_dataframe(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> pd.DataFrame:
    """
    Exécute une requête SELECT et retourne un DataFrame pandas.
    """
    return pd.read_sql(text(query), engine, params=params)


def execute_query(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> None:
    """
    Exécute une requête INSERT, UPDATE ou DELETE.
    """
    try:
        with engine.begin() as conn:  # commit automatique
            conn.execute(text(query), params or {})
    except Exception as e:
        print(f"❌ Erreur SQL : {e}")


def dataframe_to_table(
    df: pd.DataFrame,
    table_name: str,
    engine: Engine,
    schema: str = "public",
    if_exists: str = "append"
) -> None:
    """
    Insère un DataFrame pandas dans une table PostgreSQL.
    """
    df.to_sql(
        table_name,
        engine,
        schema=schema,
        if_exists=if_exists,
        index=False,
        method="multi"
    )


In [2]:
host=os.getenv("HOST")
database=os.getenv("DATABASE")
user=os.getenv("USER")
password=os.getenv("PASSWORD")


engine = create_postgres_engine(
    host=host,
    database=database,
    user=user,
    password=password
)

In [3]:
req = """
WITH winners AS (
    SELECT
        home_team,
        away_team,
        CASE
            WHEN home_result > away_result THEN home_team
            WHEN away_result > home_result THEN away_team
            ELSE 'Draw'
        END AS winner,
        CASE
            WHEN home_result > away_result THEN away_team   
            WHEN away_result > home_result THEN home_team
            ELSE 'Draw'
        END AS loser
    FROM matches)
SELECT winner, COUNT(*) AS win_count, 
       ROUND( (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM winners WHERE winner <> 'Draw'), 2) AS win_percentage
FROM winners
WHERE winner <> 'Draw'
GROUP BY winner
ORDER BY win_count DESC;
"""

In [4]:
# SELECT
df_matches = select_to_dataframe(
    engine,
    req
)
df_matches

,winner,win_count,win_percentage
0,Brazil,123,2.10
1,Argentina,117,2.00
2,Mexico,114,1.95
3,Italy,111,1.90
4,Spain,100,1.71
...,...,...,...
239,G1,1,0.02
240,Serbia,1,0.02
241,Cambodia,1,0.02
242,Denmark,1,0.02


In [9]:
req = """
SELECT
CASE WHEN home_result > away_result THEN home_team
     WHEN away_result > home_result THEN away_team
     ELSE CONCAT(home_team, ' & ', away_team)
     END AS team,
     CASE WHEN home_result > away_result THEN home_result
     WHEN away_result > home_result THEN away_result
     ELSE home_result
     END AS Nbr_buts,
     edition AS Année
FROM matches
WHERE stage = 'final'
ORDER BY Nbr_buts DESC;

"""

# SELECT
df_matches = select_to_dataframe(
    engine,
    req
)
df_matches

,team,nbr_buts,année
0,Brazil,7,1950
1,Brazil,6,1950
2,Brazil,5,1958
3,England,4,1966
4,Italy,4,1938
5,Brazil,4,1970
6,France,4,2018
7,Uruguay,4,1930
8,Argentina & France,3,2022
9,Uruguay,3,1950
